In [10]:
import os
import cv2
import pickle
import shutil
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm

In [11]:
sample_index = 1

paths = [{
    "styled_img_path": f"/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-027/diffgen-2025-06-29-03-16/styled_samples_288x256x256x4.pkl",
    "masked_images_folder_path": '/home/miguel/GI/0 - Data Exploration & Analysis/GI-Roberta/gi-roberta-dataset/full_dataset'
}, {
    "styled_img_path": f'/home/miguel/GI/1.5 - Synthetic Data Generation/diffuse-gen/diffuse-gen/image_samples_FD-032/diffgen-2025-07-08-08-03/styled_samples_3x256x256x4.pkl',
    "masked_images_folder_path": '/home/miguel/GI/0 - Data Exploration & Analysis/GI-Roberta/gi-roberta-dataset/full_dataset'
}]

styled_img_path, masked_images_folder_path = paths[sample_index]["styled_img_path"], paths[sample_index]["masked_images_folder_path"]

shape = (256, 256)

In [12]:
with open(styled_img_path, "rb") as f:
    styled_images = pickle.load(f)

In [14]:
export_path_nonzero_masks = 'nonzero_masks'
export_path_zero_masks = 'zero_masks'

In [22]:
filename_images = {}
masked_images = os.listdir(masked_images_folder_path)

for filename in masked_images:
    if not filename.endswith('.png') and not filename.endswith('.jpg'):
        continue
        
    img_path = os.path.join(masked_images_folder_path, filename)
    image = Image.open(img_path)
    filename_images[filename] = image

In [23]:
# Clear the export path directories if they exists
if os.path.exists(export_path_nonzero_masks):
    shutil.rmtree(export_path_nonzero_masks)

os.makedirs(export_path_nonzero_masks, exist_ok=True)

if os.path.exists(export_path_zero_masks):
    shutil.rmtree(export_path_zero_masks)
    

os.makedirs(export_path_zero_masks, exist_ok=True)

In [24]:
num_nonzero_masks = 0
num_masks = 0

mask_min = 0
mask_max = 0

for key, styled_list in tqdm(styled_images.items()):
    styled_image = styled_list[0]
    
    # Split synthetic image and mask
    synthetic_img = styled_image[:, :, :3].astype(np.uint8)
    mask = styled_image[:, :, 3]

    mask_min = min(mask_min, mask.flatten().min())
    mask_max = max(mask_max, mask.flatten().max())

    mask_normalized = mask / 255.0  # Normalize to [0,1] for transparency
     
    filename = key.split('/')[-1]  # Extract filename

    original_image = filename_images.get(filename, None)
    if original_image is None:
        print(f"Warning: No original image found for {filename}. Skipping.")
        continue
    
    has_mask = mask_normalized.flatten().sum() > 5
    if has_mask:
        num_nonzero_masks += 1
    num_masks += 1

    export_path = export_path_nonzero_masks if has_mask else export_path_zero_masks

    # Save original synthetic image
    orig_filename = os.path.join(export_path, f"{filename}_synthetic.png")
    cv2.imwrite(orig_filename, cv2.cvtColor(synthetic_img, cv2.COLOR_RGB2BGR))

    # Save the raw mask
    mask_filename = os.path.join(export_path, f"{filename}_synthetic_mask.png")
    cv2.imwrite(mask_filename, (mask_normalized * 255).astype(np.uint8))

    # Create and save the overlay
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
    axes[0].imshow(synthetic_img)
    axes[0].set_title(f"Synthetic: {filename}")
    axes[0].axis('off')

    axes[1].imshow(synthetic_img)
    axes[1].imshow(mask_normalized, cmap='jet', alpha=0.5)
    axes[1].set_title(f"Synthetic + Mask Overlay: {filename}")
    axes[1].axis('off')
    
    axes[2].imshow(original_image)
    axes[2].set_title(f"Original: {filename}")
    axes[2].axis('off')

    overlay_filename = os.path.join(export_path, f"{filename}_overlay.png")
    plt.tight_layout()
    plt.savefig(overlay_filename, bbox_inches='tight')
    plt.close(fig)

100%|██████████| 3/3 [00:00<00:00,  4.39it/s]


In [60]:
mask_min

0

In [61]:
mask_max

np.uint8(255)